In [ ]:
hours_diff_with_gmt = 3

In [ ]:
min_wait_hours = 1

In [ ]:
import lib_process
import config
import gzip
import lib_db
import lib_nlp
import datetime
import lib_vk
import datetime

In [ ]:
%%time
# Data from database
cur_readslave = lib_db.get_cur(staging=False, read_slave=True)
cur = lib_db.get_cur(staging=False, read_slave=False)
cur_readslave.execute("SELECT * FROM records WHERE processed = 1 ORDER BY id DESC LIMIT 5000")
list_from_db = cur_readslave.fetchall()

In [ ]:
%%time
updated = 0
updated_legacy = 0
error = 0
cnt = 0
nowdt = datetime.datetime.now()
for l in list_from_db:
    cnt += 1
    if cnt % 100 == 0:
        print(cnt)
        print('>>>', updated, updated_legacy, error)
    l = dict(l)
    updated_after_created = (l['updated_at'] - l['created_at']).total_seconds()
    now_after_source_created = (nowdt - l['source_created_at']).total_seconds() - 3600 * hours_diff_with_gmt
#     print(updated_after_created, now_after_source_created)
    if now_after_source_created > 3600 * min_wait_hours:
        if l['coverage'] == 1 or updated_after_created < 1800:
            if l['source_id'] == 0:
#                 print(l)
                reach = lib_vk.get_post_reach(l['source_record_id'])
#                 print(reach)
                if reach < 1:
                    reach = 2
                    error += 1
                updated += 1
            else:
                reach = lib_process.calc_coverage(l['id'])
                updated_legacy += 1
            cur.execute("UPDATE records SET coverage = %s WHERE id = %s", (reach, l['id']))
print('>>>', updated, updated_legacy, error)

In [ ]:
cur = lib_db.get_cur()
cnt = 0
for i in range(len(lines)):
    l = lines[i]
    cnt += 1
    if cnt % 100 == 0:
        print(cnt)
    new_coverage = lib_process.calc_coverage(l['id'])
#     print(new_coverage)
    if new_coverage > 1 and new_coverage > l['coverage']:
#         print("UPDATE records SET coverage = %s WHERE id = %s" % (new_coverage, l['id']))
        cur.execute("UPDATE records SET coverage = %s WHERE id = %s", (new_coverage, l['id']))
        lines[i]['coverage'] = new_coverage